<a href="https://colab.research.google.com/github/Esantomi/NLP/blob/master/NLP_Day7_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN의 문제점  
- 기울기 소실
- 번역에는 사용하기 힘들다.
  - 나는 점심을 먹는다.   
  - I eat lunch  
  - 각 언어별 어순 정보를 파악하기 힘들다.

# seq2seq구현하기

## LSTM Encoder

In [ ]:
import tensorflow as tf

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(enc_units)
    # return_sequences 매개변수를 기본값 False로 전달
  def call(self, x):
    print("입력 shape:",x.shape)
    
    x=self.embedding(x)
    print("Embedding Layer를 거친 shape ", x.shape)

    output = self.lstm(x)
    print("LSTM Layer의 Output Shape :", output.shape)

    return output

In [ ]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print("Vocab Size : {0}".format(vocab_size))
print("Embedding Size : {0}".format(emb_size))
print("LSTM size : {0}".format(lstm_size))
print("Batch Size : {0}".format(batch_size))
print("Sample Sequence Length : {0}\n".format(sample_seq_len))

Vocab Size : 30000
Embedding Size : 256
LSTM size : 512
Batch Size : 1
Sample Sequence Length : 3



In [ ]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

sample_output = encoder(sample_input)
# 컨텍스트 벡터로 사용한 인코더 LSTM의 최종 State값

입력 shape: (1, 3)
Embedding Layer를 거친 shape  (1, 3, 256)
LSTM Layer의 Output Shape : (1, 512)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-6.max-800x600.jpg)

## LSTM Decoder

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(dec_units, return_sequences=True)
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.softmax = tf.keras.layers.Softmax(axis= -1)

  def call(self, x, context_v):
    # 디코더의 입력 x와 인코더의 컨텍스트 벡터를 인자로 받습니다.
    print("입력 shape :", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    context_v = tf.repeat(tf.expand_dims(context_v, axis=1), repeats=x.shape[1], axis =1)
    x = tf.concat([x, context_v], axis = -1)
    print("Context Vector가 더해진 Shape :", x.shape)

    x = self.lstm(x)
    print("LSTM Layer의 Output Shape :", x.shape)

    output = self.fc(x)
    print("Decoder 최종 Output Shape :", output.shape)

    return self.softmax(output)

In [ ]:
print("vocab Size : {0}".format(vocab_size))
print("Embedding Size : {0}".format(emb_size))
print("LSTM Size : {0}".format(lstm_size))
print("Batch Size : {0}".format(batch_size))
print("Sample Sequence Length : {0}\n".format(sample_seq_len))

vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3



In [ ]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

dec_output = decoder(sample_input, sample_output)
# Decoder.call(x, context_v)를 호출

입력 shape : (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
Context Vector가 더해진 Shape : (1, 3, 768)
LSTM Layer의 Output Shape : (1, 3, 512)
Decoder 최종 Output Shape : (1, 3, 30000)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-7.max-800x600.jpg)

- Rnn에 기반한 seq2seq 모델의 두 가지 문제점
  1. 기울기 소실
  2. 하나의 고정된 벡터에 모든 정보를 압축하려다 보니 정보 손실이 발생

# 어텐션 메커니즘 (Attention Mechanism)

- 어텐션의 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(time step)마다 인코더에서의 전체 입력 문장을 다시 한번 참고한다는 점
- 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야 할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중해서 본다.

In [ ]:
dict = {"2017": "Transformer", "2018":"BERT"}

In [ ]:
print(dict["2017"])

Transformer


In [ ]:
print(dict["2018"])

BERT


![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)

Attention(Q, K, V) = Attention Value
- Query : t 시점의 디코더 셀에서의 은닉 상태
- Key : 모든 시점의 인코더 셀의 은닉 상태
- Value : 모든 시점의 인코더 셀의 은닉 상태

## 닷 프로덕트 어텐션 (Dot-Product Attention)

![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

### 1. 어텐션 스코어 구하기

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)

$$score(s_t, h_i) = S_t^T h_i $$  
$$e^t = [s_t^T h_1, ..., s_t^T h_N]$$

### 2. 소프트맥스(softmax) 함수를 통해 어텐션 분포를 구함

![](https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG)

$$a^t = softmax(e^t)$$

### 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값을 구함

![](https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG)

$$a_t = \sum_{i=1}^{N}{a_i^th_i}$$ 

### 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결

![](https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG)

### 5. 출력층 연산의 입력이 되는 st를 계산

![](https://wikidocs.net/images/page/22893/st.PNG)

$$ \tilde{s_{t}}=tanh(W_c[a_t ; s_t] + b_c)$$

### 6. st를 출력층의 입력으로 사용

$$ \hat{y_t}=Softmax(W_y\tilde{s_t}+b_y)$$

### Bahdanau Attention

- Bahdanau Attention
$$ Score_{alignment} = W * tanh(W_{decoder} * H_{decoder} + W_{encoder} * H_{encoder}) $$

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W_decoder = tf.keras.layers.Dense(units)
    self.W_encoder = tf.keras.layers.Dense(units)
    self.W_combine = tf.keras.layers.Dense(1)

  def call(self, H_encoder, H_decoder):
    print("[H_encoder] Shape :", H_encoder.shape)

    H_encoder = self.W_encoder(H_encoder)
    print("[W_encoder X H_encoder] Shape:", H_encoder.shape)

    print("\n[H_decoder] Shape: ", H_decoder.shape)
    H_decoder = tf.expand_dims(H_decoder, 1)
    H_decoder = self.W_decoder(H_decoder)

    print("[W_decoder X H_decoder] Shape:", H_decoder.shape)

    score = self.W_combine(tf.nn.tanh(H_decoder+H_encoder))
    print("[Score_alignment]Shape :", score.shape)

    attention_weights = tf.nn.softmax(score, axis = 1)
    print("\n최종 weight : \n", attention_weights.numpy())

    context_vector = attention_weights * H_decoder
    context_vector = tf.reduce_sum(context_vector, axis = 1)

    return context_vector, attention_weights

W_size = 100

print("Hidden State를 {0}차원으로 Mapping\n".format(W_size))

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[H_encoder] Shape : (1, 10, 512)
[W_encoder X H_encoder] Shape: (1, 10, 100)

[H_decoder] Shape:  (1, 512)
[W_decoder X H_decoder] Shape: (1, 1, 100)
[Score_alignment]Shape : (1, 10, 1)

최종 weight : 
 [[[0.07395881]
  [0.09009522]
  [0.06064911]
  [0.16680424]
  [0.17511545]
  [0.08980763]
  [0.08080629]
  [0.08456039]
  [0.10162464]
  [0.07657818]]]


![](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GN-4-L-9.jpg)

### Loung Attention

$$ Score(H_{target},H_{source}) = H_{target}^T * W_{combine} * H_{source})$$

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W_combine = tf.keras.layers.Dense(units)
  
  def call(self, H_encoder, H_decoder):
    print("[H_encoder] shape: ", H_encoder.shape)

    WH = self.W_combine(H_encoder)
    print("[W_encoder X H_encoder] shape :", WH.shape)

    H_decoder = tf.expand_dims(H_decoder, 1)
    alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
    print("[Score_alignment] Shape :", alignment.shape)

    attention_weights = tf.nn.softmax(alignment, axis = 1)
    print("\n최종 weight : \n", attention_weights.numpy())

    attention_weights = tf.squeeze(attention_weights, axis = -1)
    context_vector = tf.matmul(attention_weights, H_encoder)

    return context_vector, attention_weights

emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[H_encoder] shape:  (1, 10, 512)
[W_encoder X H_encoder] shape : (1, 10, 512)
[Score_alignment] Shape : (1, 10, 1)

최종 weight : 
 [[[9.9635776e-03]
  [3.5920415e-02]
  [1.1699638e-02]
  [7.3278522e-01]
  [6.3557647e-02]
  [2.2391758e-03]
  [4.2902087e-05]
  [2.4874238e-03]
  [1.5677591e-03]
  [1.3973625e-01]]]


# 양방향 LSTM과 어텐션 메커니즘 (IMDb 리뷰 데이터)

## IMDB 리뷰 데이터 전처리하기

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = vocab_size)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in x_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, x_train))/len(x_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [ ]:
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

## 바다나우 어텐션

$$score(query, key) = V^Ttanh(W_1 key + W_2 query)$$

In [ ]:
import tensorflow as tf

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): 
    #query size (batch_size, hidden size)
    # hidde_with time axis shape (batch size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention weights shape == (batch size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis =1 )

    # context_vector shape after sum == (batch size, hidden size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

## 양방향 LSTM + 어텐션 메커니즘

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [ ]:
sequence_input = Input(shape =(max_len, ), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

In [ ]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

In [ ]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [ ]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [ ]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

In [ ]:
attention = BahdanauAttention(64)  # 가중치의 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [ ]:
dense1 = Dense(20, activation='relu')(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1,activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs = output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test, y_test), verbose = 1)

Epoch 1/5
98/98 [==============================] - 543s 5s/step - loss: 0.5059 - accuracy: 0.7366 - val_loss: 0.2964 - val_accuracy: 0.8776
Epoch 2/5
98/98 [==============================] - 522s 5s/step - loss: 0.2677 - accuracy: 0.9070 - val_loss: 0.2922 - val_accuracy: 0.8791
Epoch 3/5
98/98 [==============================] - 509s 5s/step - loss: 0.2025 - accuracy: 0.9353 - val_loss: 0.3198 - val_accuracy: 0.8774
Epoch 4/5
98/98 [==============================] - 506s 5s/step - loss: 0.1789 - accuracy: 0.9446 - val_loss: 0.3713 - val_accuracy: 0.8694
Epoch 5/5
98/98 [==============================] - 503s 5s/step - loss: 0.1454 - accuracy: 0.9594 - val_loss: 0.3643 - val_accuracy: 0.8682


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 329s 421ms/step - loss: 0.3643 - accuracy: 0.8682

 테스트 정확도: 0.8682


# seq2seq with attention 스페인-영어 번역기

## 데이터 준비하기

In [ ]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

In [ ]:
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

In [ ]:
path_to_file = os.path.dirname(path_to_zip)+ "/spa-eng/spa.txt"

In [ ]:
with open(path_to_file, "r") as f:
  raw = f.read().splitlines()

print("Data Size: ", len(raw))
print("Example:")

for sen in raw[0:100][::20]: print(">>", sen)

Data Size:  118964
Example:
>> Go.	Ve.
>> Wait.	Esperen.
>> Hug me.	Abrázame.
>> No way!	¡Ni cagando!
>> Call me.	Llamame.


## 데이터 전처리 : 정제하기

In [ ]:
def preprocess_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [ ]:
eng_corpus = []
dec_corpus = []

num_examples = 30000

for pair in raw[:num_examples]:
  eng, spa = pair.split("\t")

  eng_corpus.append(preprocess_sentence(eng))
  dec_corpus.append(preprocess_sentence(spa, s_token=True, e_token=True))

print("English :", eng_corpus[100])
print("Spanish :", dec_corpus[100])

English : go away !
Spanish : <start> salga de aqu ! <end>


## 데이터 전처리 : 토큰화

In [ ]:
def tokenize(corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  tokenizer.fit_on_texts(corpus)

  tensor = tokenizer.texts_to_sequences(corpus)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, tokenizer

In [ ]:
# 토큰화 하기
# 훈련 데이터와 검증 데이터로 분리하기
# 정제된 텍스트를 아래 tokenize() 함수를 사용해 토큰화해서 텐서로 변환하세요~

enc_tensor, enc_tokenizer = tokenize(eng_corpus)
dec_tensor, dec_tokenizer = tokenize(dec_corpus)

In [ ]:
# 그리고 변환된 텐서를 80%의 훈련데이터와 20% 검증데이터로 분리하세요
# 단 Tokenizer의 단어 수는 자유롭게 진행하세요!

enc_train, enc_val, dec_train, dec_val = train_test_split(enc_tensor, dec_tensor, test_size = 0.2)

In [ ]:
# english vocab size반환
# spanish vocab size반환

print('English Vocab Size :',len(enc_tokenizer.index_word))
print('Spanish Vocab Size :',len(dec_tokenizer.index_word))

English Vocab Size : 4931
Spanish Vocab Size : 8893


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.w_dec = tf.keras.layers.Dense(units)
    self.w_enc = tf.keras.layers.Dense(units)
    self.w_com = tf.keras.layers.Dense(1)

  def call(self, h_enc, h_dec):
    # h_enc shape : [batch x length x units]
    # h_dec shape : [batch x units]

    h_enc = self.w_enc(h_enc)
    h_dec = tf.expand_dims(h_dec, 1)
    h_dec = self.w_dec(h_dec)

    score = self.w_com(tf.nn.tanh(h_dec + h_enc))

    attn = tf.nn.softmax(score, axis =1)

    context_vec = attn * h_enc
    context_vec = tf.reduce_sum(context_vec, axis=1)
    return context_vec, attn

![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-P-2.max-800x600.jpg)

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    # todo
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(enc_units, return_sequences=True)

  def call(self, x):
    # todo 
    out = self.embedding(x)
    out = self.gru(out)

    return out

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    # To do
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(dec_units, return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, h_dec, enc_out):
    # To do
    context_vec, attn = self.attention(enc_out, h_dec)
    
    out = self.embedding(x)
    out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)

    out, h_dec = self.gru(out)
    out = tf.reshape(out, (-1, out.shape[2]))
    out = self.fc(out)

    return out, h_dec, attn

In [ ]:
BATCH_SIZE = 64
src_vocab_size = len(enc_tokenizer.index_word) + 1
tgt_vocab_size = len(dec_tokenizer.index_word) + 1

units = 1024
embedding_dim = 512

encoder = Encoder(src_vocab_size, embedding_dim, units)
decoder = Decoder(tgt_vocab_size, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))
sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_state, sample_output)

print('Decoder output :', sample_logits.shape)
print('Decoder Hidden State :', h_dec.shape)
print('Attention :', attn.shape)

Encoder Output: (64, 30, 1024)
Decoder output : (64, 8894)
Decoder Hidden State : (64, 1024)
Attention : (64, 30, 1)


## 훈련하기 1. Optimizer & loss

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
# Categorical Crossentropy()
# [0.1, 0.2, 0.7] --> onehot encoding [0, 0, 1]
# SparseCategoricalCrossentropy()
# [0.1, 0.2, 0.7] ---> 정수 인덱스 2
# True --> 모델의 출력값을 그대로 전달

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  return tf.reduce_mean(loss)

## 훈련하기 2. train_step

In [ ]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
  bsz = src.shape[0]
  loss = 0

  with tf.GradientTape() as tape:
    enc_out = encoder(src)
    h_dec = enc_out[:, -1]

    dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

    for t in range(1, tgt.shape[1]):
      pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

      loss += loss_function(tgt[:, t], pred)
      dec_src = tf.expand_dims(tgt[:,t], 1)

  batch_loss = (loss/int(tgt.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

- train_step 학습과정
  1. Encoder에 소스 문장을 전달해 컨텍스트 벡터인 enc_out을 생성
  2. Decoder에 입력으로 전달할 <start>토큰 문장 생성
  3. t=0일때, Decoder의 Hidden State는 Encoder의 Final State로 정의. h_dec = enc_out[:, -1]
  4. <start>문장과 enc_out, Hidden State를 기반으로 다음단어 (t=1)예측. pred
  5. 예측된 단어와 정답 간의 Loss를 구한 후, t=1의 정답 단어를 다음 입력으로 사용 (예측단어X)
  6. 반복!

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm
import random

epochs = 10

for epoch in range(epochs):
  total_loss = 0

  idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
  random.shuffle(idx_list)
  t = tqdm(idx_list)

  for (batch, idx) in enumerate(t):
    batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                            dec_train[idx:idx+BATCH_SIZE],
                            encoder,
                            decoder,
                            optimizer,
                            dec_tokenizer)
    total_loss += batch_loss

    t.set_description_str('Epoch %2d' % (epoch+1))
    t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch+1)))

Epoch 10: 100%|██████████| 375/375 [27:54<00:00,  4.47s/it, Loss 0.1404]


## Evaluation step

In [ ]:
@tf.function
def eval_step(src, tgt, encoder, decoder, dec_tok):
    bsz = src.shape[0]
    loss = 0

    enc_out = encoder(src)

    h_dec = enc_out[:, -1]
    
    dec_src = tf.expand_dims([dec_tok.word_index['']] * bsz, 1)

    for t in range(1, tgt.shape[1]):
        pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

        loss += loss_function(tgt[:, t], pred)
        dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))
    
    return batch_loss


# Training Process

from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                dec_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    test_loss = 0
    
    idx_list = list(range(0, enc_val.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (test_batch, idx) in enumerate(t):
        test_batch_loss = eval_step(enc_val[idx:idx+BATCH_SIZE],
                                    dec_val[idx:idx+BATCH_SIZE],
                                    encoder,
                                    decoder,
                                    dec_tokenizer)
    
        test_loss += test_batch_loss

        t.set_description_str('Test Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Test Loss %.4f' % (test_loss.numpy() / (test_batch + 1)))

  0%|          | 0/94 [00:00<?, ?it/s]

KeyError: ignored

## Attention Map 시각화

In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))


translate("Can I have some coffee?", encoder, decoder)

![](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GN-4-P-3.jpg)

# 주말 동안 공부해야 할 숙제

한-영 번역기 만들기
한-영 번역기 만들기
1. 데이터 다운로드
- 데이터 : https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1
- korean-english-park.train.tar.gz
2. 데이터 정제
- set 데이터형이 중복이 허용하지 않다는 것을 활용해 중복된 데이터를 제거
  - 데이터 병렬 쌍이 흐트러지지 않게 주의!
  - cleaned_corpus에 저장
- 앞서 정의한 preprocessing()함수는 한글에 대해 동작하지 않아요.
  - 한글에 적용할 수 있는 정규식을 추가해여 함수를 재정의 하세요.
- 타겟 언어인 영문엔 <start>토큰과 <end>토큰을 추가하고 split()함수로 토큰화 합니다. 한글 토큰화는 konlpy의 mecab클래스를 사용합니다.
  - cleaned_corpus로부터 토큰의길이가 40이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 각각 구축하기

3. 토큰화
- tokenize()함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻으세요!
  - 단어수는 실험을 통해 적당한 값을 맞춰줍시다(최소 10000이상!)
4. 훈련하기

In [ ]:
```
sudo apt -qq -y install fonts-nanum
```

In [ ]:
```
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()
```